In this notebook I applied two of the best know Deep Learning Models LSTM and CNN. I wanted to see if they can outperform the Machine Learning Algoriths I used in the previous notebook

In [ ]:

import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding,GRU
from keras.preprocessing import sequence
from keras.preprocessing import text
from keras.layers import Dropout, Activation
from keras.layers import Conv1D, Flatten
from keras import metrics
from keras.utils import to_categorical
from keras.layers import GlobalMaxPooling1D, Input, MaxPooling1D
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score,precision_score, precision_recall_fscore_support


MAX_WORDS_IN_SEQ = 1000
EMBED_DIM = 100

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving twitter_data.pkl to twitter_data.pkl


In [ ]:
df = pd.read_pickle("./twitter_data.pkl")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        30000 non-null  int64         
 1   text      30000 non-null  object        
 2   date      30000 non-null  datetime64[ns]
 3   source    30000 non-null  object        
 4   likes     30000 non-null  int64         
 5   retweets  30000 non-null  int64         
 6   class     30000 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 1.6+ MB


,id,text,date,source,likes,retweets,class
0,1288573860627877888,A medical group is warning that without action...,2020-07-29 20:35:23,TweetDeck,23,14,0
1,1288572426771554305,Covid-19 is now one of the leading causes of d...,2020-07-29 20:29:41,TweetDeck,91,47,0
2,1288570544481345538,"JUST IN: More than 150,000 people have died in...",2020-07-29 20:22:12,SocialFlow,242,145,0
3,1288566498995380232,This woman went to the hospital after breaking...,2020-07-29 20:06:08,SocialFlow,379,63,0
4,1288562923904724992,President Trump’s tweet on dismantling an Obam...,2020-07-29 19:51:56,Twitter Media Studio,473,164,0


Here I did the same thing as the previous notebook. I seperated the tweets related to the corona virus from the other tweets. In order to test the models on them while training it on the general tweets. 

In [ ]:
corona_tweets = list()
for i in range(0, len(df)):
    t=df['text'][i]
    t=t.lower()
    if t.find('corona')!=-1 or t.find('covid')!=-1: 
        corona_tweets.append([df['id'][i],
                         df['text'][i],
                         df['date'][i],
                         df['source'][i],
                         df['likes'][i],
                         df['retweets'][i],
                         df['class'][i]
                         ])
        df=df.drop(i)
df.reset_index(drop=True,inplace=True)
df_ct= pd.DataFrame(corona_tweets,columns = ['id' , 'text', 'date','source','likes','retweets','class'])
df_ct.info()   
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3949 entries, 0 to 3948
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        3949 non-null   int64         
 1   text      3949 non-null   object        
 2   date      3949 non-null   datetime64[ns]
 3   source    3949 non-null   object        
 4   likes     3949 non-null   int64         
 5   retweets  3949 non-null   int64         
 6   class     3949 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 216.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26051 entries, 0 to 26050
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        26051 non-null  int64         
 1   text      26051 non-null  object        
 2   date      26051 non-null  datetime64[ns]
 3   source    26051 non-null  object        
 4   likes     26051 non-

In [ ]:
#Shuffling the Data
df = df.sample(frac=1).reset_index(drop=True)
df_ct = df_ct.sample(frac=1).reset_index(drop=True)


In [ ]:
y_train = df['class'].values
y_test = df_ct['class'].values


In [ ]:
X_train = df['text'].values
X_test = df_ct['text'].values

In [ ]:
#This class allows to vectorize a text corpus, by turning each text into a sequence of integers 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
#This function transforms a list of sequences into a 2D Numpy array of shape 
X_train_pad = pad_sequences(X_train_seq, maxlen=20, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=20, padding='post')


In [ ]:
laenge_pads = 20
anz_woerter = 7982

lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=64551, output_dim=20, input_length=laenge_pads))
lstm_model.add(Dropout(0.8))
lstm_model.add(LSTM(400))
lstm_model.add(Dropout(0.86))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 20)            1291020   
_________________________________________________________________
dropout (Dropout)            (None, 20, 20)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 400)               673600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 401       
Total params: 1,965,021
Trainable params: 1,965,021
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history_lstm = lstm_model.fit(X_train_pad, y_train, epochs=10, batch_size=64, 
                        validation_data=(X_test_pad, y_test))

Epoch 1/10
408/408 [==============================] - 79s 193ms/step - loss: 0.4913 - accuracy: 0.7574 - val_loss: 0.3074 - val_accuracy: 0.8888
Epoch 2/10
408/408 [==============================] - 78s 191ms/step - loss: 0.3061 - accuracy: 0.8765 - val_loss: 0.2592 - val_accuracy: 0.8939
Epoch 3/10
408/408 [==============================] - 79s 194ms/step - loss: 0.2316 - accuracy: 0.9122 - val_loss: 0.2049 - val_accuracy: 0.9250
Epoch 4/10
408/408 [==============================] - 78s 191ms/step - loss: 0.1859 - accuracy: 0.9318 - val_loss: 0.2187 - val_accuracy: 0.9162
Epoch 5/10
408/408 [==============================] - 80s 196ms/step - loss: 0.1402 - accuracy: 0.9487 - val_loss: 0.2011 - val_accuracy: 0.9385
Epoch 6/10
408/408 [==============================] - 78s 192ms/step - loss: 0.1109 - accuracy: 0.9607 - val_loss: 0.2294 - val_accuracy: 0.9331
Epoch 7/10
408/408 [==============================] - 79s 193ms/step - loss: 0.0880 - accuracy: 0.9701 - val_loss: 0.2805 - val_ac

In [ ]:
acc = lstm_model.evaluate(X_test_pad, y_test)
print("Test loss is {0:.2f} accuracy is {1:.2f} ".format(acc[0],acc[1]))

124/124 [==============================] - 4s 30ms/step - loss: 0.2030 - accuracy: 0.9210
Test loss is 0.20 accuracy is 0.92 


In [ ]:
pred= lstm_model.predict(X_test_pad)
for i in range(0, len(pred)):
  if pred[i][0] >= 0.5:
    pred[i][0]=1
  if pred[i][0]<0.5:
    pred[i][0]=0


In [ ]:
print(classification_report(y_test,pred))
precision =precision_score(y_test, pred, average='binary')
print()
print('Confusion Matrix: \n',confusion_matrix(y_test,pred))
print()
print("Overall Precision, Recall, F1-score :",precision_recall_fscore_support(y_test, pred, average='weighted'))


              precision    recall  f1-score   support

           0       0.98      0.93      0.95      3434
           1       0.64      0.89      0.75       515

    accuracy                           0.92      3949
   macro avg       0.81      0.91      0.85      3949
weighted avg       0.94      0.92      0.93      3949


Confusion Matrix: 
 [[3177  257]
 [  55  460]]

Overall Precision, Recall, F1-score : (0.9384570683819818, 0.920992656368701, 0.9262726384782601, None)


In [ ]:
gru1 = Sequential()
gru1.add(Embedding(input_dim=64551, output_dim=20, input_length=laenge_pads))
gru1.add(Dropout(0.8))
gru1.add(GRU(140, return_sequences=False))
gru1.add(Dropout(0.86))
gru1.add(Dense(1, activation='sigmoid', name='Classification'))
gru1.summary()

gru1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 20)            1291020   
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 20)            0         
_________________________________________________________________
gru (GRU)                    (None, 140)               68040     
_________________________________________________________________
dropout_3 (Dropout)          (None, 140)               0         
_________________________________________________________________
Classification (Dense)       (None, 1)                 141       
Total params: 1,359,201
Trainable params: 1,359,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history_gru = gru1.fit(X_train_pad, y_train, epochs=10, batch_size=64, validation_data=(X_test_pad, y_test))

Epoch 1/10
408/408 [==============================] - 21s 51ms/step - loss: 0.5133 - accuracy: 0.7325 - val_loss: 0.2453 - val_accuracy: 0.9101
Epoch 2/10
408/408 [==============================] - 21s 50ms/step - loss: 0.2966 - accuracy: 0.8791 - val_loss: 0.2248 - val_accuracy: 0.9263
Epoch 3/10
408/408 [==============================] - 21s 50ms/step - loss: 0.2241 - accuracy: 0.9147 - val_loss: 0.2201 - val_accuracy: 0.9174
Epoch 4/10
408/408 [==============================] - 21s 51ms/step - loss: 0.1793 - accuracy: 0.9333 - val_loss: 0.2300 - val_accuracy: 0.9180
Epoch 5/10
408/408 [==============================] - 21s 51ms/step - loss: 0.1397 - accuracy: 0.9512 - val_loss: 0.3261 - val_accuracy: 0.8863
Epoch 6/10
408/408 [==============================] - 21s 50ms/step - loss: 0.1063 - accuracy: 0.9611 - val_loss: 0.3125 - val_accuracy: 0.8979
Epoch 7/10
408/408 [==============================] - 21s 51ms/step - loss: 0.0855 - accuracy: 0.9711 - val_loss: 0.2145 - val_accuracy:

In [ ]:
acc = gru1.evaluate(X_test_pad, y_test)
print("Test loss is {0:.2f} accuracy is {1:.2f} ".format(acc[0],acc[1]))

124/124 [==============================] - 1s 5ms/step - loss: 0.2797 - accuracy: 0.9154
Test loss is 0.28 accuracy is 0.92 


In [ ]:
pred= gru1.predict(X_test_pad)
for i in range(0, len(pred)):
  if pred[i][0] >= 0.5:
    pred[i][0]=1
  if pred[i][0]<0.5:
    pred[i][0]=0

In [ ]:
print(classification_report(y_test,pred))
precision =precision_score(y_test, pred, average='binary')
print()
print('Confusion Matrix: \n',confusion_matrix(y_test,pred))
print()
print("Overall Precision, Recall, F1-score :",precision_recall_fscore_support(y_test, pred, average='weighted'))


              precision    recall  f1-score   support

           0       0.98      0.92      0.95      3434
           1       0.62      0.89      0.73       515

    accuracy                           0.92      3949
   macro avg       0.80      0.90      0.84      3949
weighted avg       0.93      0.92      0.92      3949


Confusion Matrix: 
 [[3159  275]
 [  59  456]]

Overall Precision, Recall, F1-score : (0.9349957777133822, 0.9154216257280324, 0.9213794658810971, None)


As you can see, applying LSTM gave a test accuracy of 92%.<br>
Next I will apply CNN to see if it can give us a better accuracy

In [ ]:
cnn_model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
cnn_model.add(Embedding(64551, 100, input_length=1000))
cnn_model.add(Dropout(0.2))
cnn_model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(1))
cnn_model.add(Activation('sigmoid'))
cnn_model.summary()
cnn_model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 100)         6455100   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 998, 64)           19264     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 6,474,429
Trainable params: 6,474,429
Non-trainable params: 0
____________________________________________

In [ ]:
X_train_pad = pad_sequences(X_train_seq, maxlen=1000, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=1000, padding='post')


In [ ]:
history_cnn = cnn_model.fit(X_train_pad,
    y_train,
    batch_size=64,
    epochs=10,
    validation_data=(X_test_pad, y_test))

Epoch 1/10
408/408 [==============================] - 141s 346ms/step - loss: 0.3707 - acc: 0.8239 - val_loss: 0.2298 - val_acc: 0.9025
Epoch 2/10
408/408 [==============================] - 142s 347ms/step - loss: 0.0888 - acc: 0.9692 - val_loss: 0.2566 - val_acc: 0.9061
Epoch 3/10
408/408 [==============================] - 141s 346ms/step - loss: 0.0145 - acc: 0.9966 - val_loss: 0.2799 - val_acc: 0.9035
Epoch 4/10
408/408 [==============================] - 143s 352ms/step - loss: 0.0019 - acc: 0.9999 - val_loss: 0.2643 - val_acc: 0.9071
Epoch 5/10
408/408 [==============================] - 140s 344ms/step - loss: 5.8305e-04 - acc: 1.0000 - val_loss: 0.2905 - val_acc: 0.9025
Epoch 6/10
408/408 [==============================] - 142s 348ms/step - loss: 2.9348e-04 - acc: 1.0000 - val_loss: 0.2953 - val_acc: 0.9007
Epoch 7/10
408/408 [==============================] - 143s 351ms/step - loss: 1.6775e-04 - acc: 1.0000 - val_loss: 0.2993 - val_acc: 0.9002
Epoch 8/10
408/408 [================

In [ ]:
acc = cnn_model.evaluate(X_test_pad, y_test)
print("Test loss is {0:.2f} accuracy is {1:.2f} ".format(acc[0],acc[1]))

124/124 [==============================] - 3s 27ms/step - loss: 0.3141 - acc: 0.8947
Test loss is 0.31 accuracy is 0.89 


In [ ]:
pred= cnn_model.predict(X_test_pad)
for i in range(0, len(pred)):
  if pred[i][0] >= 0.5:
    pred[i][0]=1
  if pred[i][0]<0.5:
    pred[i][0]=0

In [ ]:
print(classification_report(y_test,pred))
precision =precision_score(y_test, pred, average='binary')
print()
print('Confusion Matrix: \n',confusion_matrix(y_test,pred))
print()
print("Overall Precision, Recall, F1-score :",precision_recall_fscore_support(y_test, pred, average='weighted'))


              precision    recall  f1-score   support

           0       0.98      0.90      0.94      3434
           1       0.56      0.89      0.69       515

    accuracy                           0.89      3949
   macro avg       0.77      0.89      0.81      3949
weighted avg       0.93      0.89      0.90      3949


Confusion Matrix: 
 [[3076  358]
 [  58  457]]

Overall Precision, Recall, F1-score : (0.9266212030624716, 0.8946568751582679, 0.9041318521700897, None)
